In [1]:
# Фрагменты модуля, посвященного алгоритмам поиска, из
# официального репозитория AIMA (https://github.com/aimacode/aima-python)
import search

# Головоломка "игра в 8"

Определим правила (состояние, возможные переходы и пр.) головоломки "игра в 8" с помощью примитивов, определенных в репозитории AIMA, чтобы получить возможность применять реализованные в нем алгоритмы поиска.

Определение задачи, решаемой с помощью данной реализации поисковых алгоритмов, предполагает создание класса, реализующего следующие операции:

- `actions(state)` - Iterable по действиям, допустимым в заданном состоянии;
- `result(state, action)` - новое состояние, которое получается при применении заданного действия в заданном состоянии;
- `action_cost(s, a, s1)` - стоимость совершения заданного действия в заданном состоянии;
- `h(node)` - значение эвристики для заданного узла дерева поиска (используется при информированном поиске).

В экземляре задачи должны быть также поля `initial` и `goal`, соответствующие начальному и целевому состояниям.

В модуле определен класс `Problem`, от которого целесообразно наследовать все подобные определения задач. В данном классе, в частности, предлагаются разумные значения по умолчанию для стоимости действия (все действия считаются имеющими одинаковую стоимость - единица) и эвристики (0 - соответствует неинформативной эвристике).

Состояние и действие (параметры `state` и `action`) могут иметь то представление, которое удобно для данной задачи, алгоритмы поиска никак не пытаются их интерпретировать - вся обработка происходит только внутри реализации класса задачи. Параметр `node`, передаваемый в функцию `h`, соответствует структуре `search.Node` (узел дерева поиска):

- `state` - состояние;
- `parent` - родительское состояние;
- `action` - действие, в результате которого был осуществлен переход из `parent` в `state`;
- `path_cost` - стоимость пути (от начального состояния до `state`), соответствущего данной ветви дерева.


In [2]:
class EightPuzzle(search.Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board,
    where one of the squares is a blank, trying to reach a goal configuration.
    A board state is represented as a tuple of length 9, where the element at index i 
    represents the tile number at index i, or 0 if for the empty square, e.g. the goal:
        1 2 3
        4 5 6 ==> (1, 2, 3, 4, 5, 6, 7, 8, 0)
        7 8 _
    """

    def __init__(self, initial, goal=(0, 1, 2, 3, 4, 5, 6, 7, 8)):
        self.initial, self.goal = initial, goal
    
    def actions(self, state):
        """The indexes of the squares that the blank can move to."""
        moves = ((1, 3),    (0, 2, 4),    (1, 5),
                 (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
                 (3, 7),    (4, 6, 8),    (7, 5))
        blank = state.index(0)
        return moves[blank]
    
    def result(self, state, action):
        """Swap the blank with the square numbered `action`."""
        s = list(state)
        blank = state.index(0)
        s[action], s[blank] = s[blank], s[action]
        return tuple(s)
    

In [3]:
p1 = EightPuzzle((0, 1, 2, 3, 4, 5, 6, 7, 8))
print(p1)

EightPuzzle((0, 1, 2, 3, 4, 5, 6, 7, 8), (0, 1, 2, 3, 4, 5, 6, 7, 8))


## Визуализация

In [4]:
def board8(board, fmt=(3 * '{} {} {}\n')):
    "A string representing an 8-puzzle board"
    return fmt.format(*board).replace('0', '_')

In [5]:
print(board8(p1.initial))

_ 1 2
3 4 5
6 7 8



# Экспериментальная оценка сложности алгоритмов поиска

In [6]:
e1 = EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8))
e2 = EightPuzzle((1, 2, 3, 4, 5, 6, 7, 8, 0))
e3 = EightPuzzle((4, 0, 2, 5, 1, 3, 7, 8, 6))
e4 = EightPuzzle((7, 2, 4, 5, 0, 6, 8, 3, 1))
e5 = EightPuzzle((8, 6, 7, 2, 5, 4, 3, 0, 1))

## Неинформированный поиск

In [7]:
search.report([search.uniform_cost_search], [e1, e2, e3, e4, e5])

uniform_cost_search:
      124 nodes |       46 goal |    5 cost |      50 actions | EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8),
  214,952 nodes |   79,187 goal |   22 cost |  79,208 actions | EightPuzzle((1, 2, 3, 4, 5, 6, 7, 8, 0),
  300,925 nodes |  112,082 goal |   23 cost | 112,104 actions | EightPuzzle((4, 0, 2, 5, 1, 3, 7, 8, 6),
  457,766 nodes |  171,571 goal |   26 cost | 171,596 actions | EightPuzzle((7, 2, 4, 5, 0, 6, 8, 3, 1),
  466,441 nodes |  174,474 goal |   27 cost | 174,500 actions | EightPuzzle((8, 6, 7, 2, 5, 4, 3, 0, 1),
1,440,208 nodes |  537,360 goal |  103 cost | 537,458 actions | TOTAL



## Информированный поиск

In [8]:
def hamming_distance(A, B):
    "Number of positions where vectors A and B are different."
    return sum(a != b for a, b in zip(A, B))

def h1(problem, node):
    """The misplaced tiles heuristic."""
    return hamming_distance(node.state, problem.goal)

def h2(problem, node):
    """The Manhattan heuristic."""
    X = (0, 1, 2, 0, 1, 2, 0, 1, 2)
    Y = (0, 0, 0, 1, 1, 1, 2, 2, 2)
    return sum(abs(X[s] - X[g]) + abs(Y[s] - Y[g])
               for (s, g) in zip(node.state, problem.goal) if s != 0)

In [9]:
def astar_misplaced_tiles(problem): return search.astar_search(problem, h=lambda n: h1(problem, n))
def astar_manhattan_tiles(problem): return search.astar_search(problem, h=lambda n: h2(problem, n))

In [10]:
search.report([astar_misplaced_tiles,
               astar_manhattan_tiles],
              [e1, e2, e3, e4, e5])

astar_misplaced_tiles:
       17 nodes |        7 goal |    5 cost |      11 actions | EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8),
   23,407 nodes |    8,726 goal |   22 cost |   8,747 actions | EightPuzzle((1, 2, 3, 4, 5, 6, 7, 8, 0),
   38,632 nodes |   14,433 goal |   23 cost |  14,455 actions | EightPuzzle((4, 0, 2, 5, 1, 3, 7, 8, 6),
  124,324 nodes |   46,553 goal |   26 cost |  46,578 actions | EightPuzzle((7, 2, 4, 5, 0, 6, 8, 3, 1),
  156,111 nodes |   58,475 goal |   27 cost |  58,501 actions | EightPuzzle((8, 6, 7, 2, 5, 4, 3, 0, 1),
  342,491 nodes |  128,194 goal |  103 cost | 128,292 actions | TOTAL

astar_manhattan_tiles:
       15 nodes |        6 goal |    5 cost |      10 actions | EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8),
    3,614 nodes |    1,349 goal |   22 cost |   1,370 actions | EightPuzzle((1, 2, 3, 4, 5, 6, 7, 8, 0),
    5,373 nodes |    2,010 goal |   23 cost |   2,032 actions | EightPuzzle((4, 0, 2, 5, 1, 3, 7, 8, 6),
   10,832 nodes |    4,086 goal |   26 cost